In [2]:
# For tips on running notebooks in Google Colab, see
# https://pytorch.org/tutorials/beginner/colab
%matplotlib inline

[Learn the Basics](intro.html) \|\|
[Quickstart](quickstart_tutorial.html) \|\|
[Tensors](tensorqs_tutorial.html) \|\| [Datasets &
DataLoaders](data_tutorial.html) \|\|
[Transforms](transforms_tutorial.html) \|\| **Build Model** \|\|
[Autograd](autogradqs_tutorial.html) \|\|
[Optimization](optimization_tutorial.html) \|\| [Save & Load
Model](saveloadrun_tutorial.html)

Build the Neural Network
========================

Neural networks comprise of layers/modules that perform operations on
data. The [torch.nn](https://pytorch.org/docs/stable/nn.html) namespace
provides all the building blocks you need to build your own neural
network. Every module in PyTorch subclasses the
[nn.Module](https://pytorch.org/docs/stable/generated/torch.nn.Module.html).
A neural network is a module itself that consists of other modules
(layers). This nested structure allows for building and managing complex
architectures easily.

In the following sections, we\'ll build a neural network to classify
images in the FashionMNIST dataset.


In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

Get Device for Training
=======================

We want to be able to train our model on an
[accelerator](https://pytorch.org/docs/stable/torch.html#accelerators)
such as CUDA, MPS, MTIA, or XPU. If the current accelerator is
available, we will use it. Otherwise, we use the CPU.


In [3]:
# Check PyTorch version
print(f"PyTorch version: {torch.__version__}")

# Check for CUDA availability
if torch.cuda.is_available():
    device = "cuda"
else:
    device = "cpu"

print(f"Using {device} device")

PyTorch version: 2.5.1+cu124
Using cuda device


Define the Class
================

We define our neural network by subclassing `nn.Module`, and initialize
the neural network layers in `__init__`. Every `nn.Module` subclass
implements the operations on input data in the `forward` method.


In [4]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

We create an instance of `NeuralNetwork`, and move it to the `device`,
and print its structure.


In [5]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


To use the model, we pass it the input data. This executes the model\'s
`forward`, along with some [background
operations](https://github.com/pytorch/pytorch/blob/270111b7b611d174967ed204776985cefca9c144/torch/nn/modules/module.py#L866).
Do not call `model.forward()` directly!

Calling the model on the input returns a 2-dimensional tensor with dim=0
corresponding to each output of 10 raw predicted values for each class,
and dim=1 corresponding to the individual values of each output. We get
the prediction probabilities by passing it through an instance of the
`nn.Softmax` module.


In [6]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

Predicted class: tensor([1], device='cuda:0')


------------------------------------------------------------------------


Model Layers
============

Let\'s break down the layers in the FashionMNIST model. To illustrate
it, we will take a sample minibatch of 3 images of size 28x28 and see
what happens to it as we pass it through the network.


In [7]:
input_image = torch.rand(3,28,28)
print(input_image.size())

torch.Size([3, 28, 28])


In [9]:
input_image

tensor([[[0.1041, 0.6642, 0.8865,  ..., 0.8074, 0.8649, 0.8913],
         [0.4938, 0.8680, 0.0176,  ..., 0.0788, 0.0485, 0.2139],
         [0.2456, 0.4244, 0.0550,  ..., 0.7351, 0.1529, 0.0170],
         ...,
         [0.1616, 0.1474, 0.1134,  ..., 0.1225, 0.7064, 0.4088],
         [0.8248, 0.5359, 0.7447,  ..., 0.3934, 0.8279, 0.2435],
         [0.4034, 0.7625, 0.8573,  ..., 0.1563, 0.7311, 0.9676]],

        [[0.6070, 0.7472, 0.6492,  ..., 0.4927, 0.6083, 0.6294],
         [0.4802, 0.0917, 0.7332,  ..., 0.3275, 0.4717, 0.2612],
         [0.6274, 0.1361, 0.7011,  ..., 0.0664, 0.0216, 0.7632],
         ...,
         [0.2861, 0.3494, 0.7235,  ..., 0.3752, 0.3210, 0.7148],
         [0.3772, 0.2052, 0.3908,  ..., 0.2823, 0.0867, 0.9238],
         [0.9481, 0.7678, 0.1393,  ..., 0.0518, 0.0048, 0.3180]],

        [[0.7604, 0.0543, 0.5064,  ..., 0.6290, 0.3314, 0.3878],
         [0.1690, 0.9217, 0.2242,  ..., 0.2452, 0.0972, 0.7110],
         [0.6054, 0.1016, 0.3727,  ..., 0.2637, 0.7226, 0.

nn.Flatten
==========

We initialize the
[nn.Flatten](https://pytorch.org/docs/stable/generated/torch.nn.Flatten.html)
layer to convert each 2D 28x28 image into a contiguous array of 784
pixel values ( the minibatch dimension (at dim=0) is maintained).


In [8]:
flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [10]:
flat_image

tensor([[0.1041, 0.6642, 0.8865,  ..., 0.1563, 0.7311, 0.9676],
        [0.6070, 0.7472, 0.6492,  ..., 0.0518, 0.0048, 0.3180],
        [0.7604, 0.0543, 0.5064,  ..., 0.8392, 0.7487, 0.6078]])

nn.Linear
=========

The [linear
layer](https://pytorch.org/docs/stable/generated/torch.nn.Linear.html)
is a module that applies a linear transformation on the input using its
stored weights and biases.


In [11]:
layer1 = nn.Linear(in_features=28*28, out_features=20)
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


In [12]:
hidden1

tensor([[-0.2635,  0.4589, -0.0954,  0.1174, -0.1187,  0.3627,  0.2919, -0.1415,
          0.2764,  0.2747, -0.0103,  0.3550, -0.0704, -0.1993, -0.0670, -0.3247,
         -0.6411, -0.0427,  0.5635, -0.0960],
        [-0.0637,  0.4324, -0.3145,  0.1958,  0.2147,  0.6603,  0.8161,  0.1342,
         -0.1915, -0.0595,  0.0862, -0.2509, -0.2520,  0.0324, -0.4304, -0.1478,
         -0.2823,  0.1197,  0.5451,  0.2152],
        [-0.0463,  0.4023,  0.2032,  0.4113,  0.0776,  0.5982,  0.4663,  0.0033,
          0.1075,  0.1920,  0.0586, -0.2193, -0.3082, -0.3627, -0.1487, -0.1938,
         -0.6051,  0.0566,  0.3677, -0.0866]], grad_fn=<AddmmBackward0>)

nn.ReLU
=======

Non-linear activations are what create the complex mappings between the
model\'s inputs and outputs. They are applied after linear
transformations to introduce *nonlinearity*, helping neural networks
learn a wide variety of phenomena.

In this model, we use
[nn.ReLU](https://pytorch.org/docs/stable/generated/torch.nn.ReLU.html)
between our linear layers, but there\'s other activations to introduce
non-linearity in your model.


In [13]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[-0.2635,  0.4589, -0.0954,  0.1174, -0.1187,  0.3627,  0.2919, -0.1415,
          0.2764,  0.2747, -0.0103,  0.3550, -0.0704, -0.1993, -0.0670, -0.3247,
         -0.6411, -0.0427,  0.5635, -0.0960],
        [-0.0637,  0.4324, -0.3145,  0.1958,  0.2147,  0.6603,  0.8161,  0.1342,
         -0.1915, -0.0595,  0.0862, -0.2509, -0.2520,  0.0324, -0.4304, -0.1478,
         -0.2823,  0.1197,  0.5451,  0.2152],
        [-0.0463,  0.4023,  0.2032,  0.4113,  0.0776,  0.5982,  0.4663,  0.0033,
          0.1075,  0.1920,  0.0586, -0.2193, -0.3082, -0.3627, -0.1487, -0.1938,
         -0.6051,  0.0566,  0.3677, -0.0866]], grad_fn=<AddmmBackward0>)


After ReLU: tensor([[0.0000, 0.4589, 0.0000, 0.1174, 0.0000, 0.3627, 0.2919, 0.0000, 0.2764,
         0.2747, 0.0000, 0.3550, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.5635, 0.0000],
        [0.0000, 0.4324, 0.0000, 0.1958, 0.2147, 0.6603, 0.8161, 0.1342, 0.0000,
         0.0000, 0.0862, 0.0000, 0.0000, 0.0324, 0.00

nn.Sequential
=============

[nn.Sequential](https://pytorch.org/docs/stable/generated/torch.nn.Sequential.html)
is an ordered container of modules. The data is passed through all the
modules in the same order as defined. You can use sequential containers
to put together a quick network like `seq_modules`.


In [14]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)

In [15]:
logits

tensor([[ 0.1028, -0.0718,  0.1457, -0.1696,  0.0377, -0.2148,  0.0056, -0.3730,
         -0.2826,  0.0219],
        [ 0.1049, -0.0349,  0.1525, -0.1262,  0.0204, -0.1997, -0.0188, -0.3334,
         -0.2156,  0.0603],
        [ 0.1962, -0.1181,  0.2968, -0.0483, -0.0103, -0.2684, -0.0864, -0.3283,
         -0.1928,  0.1222]], grad_fn=<AddmmBackward0>)

nn.Softmax
==========

The last linear layer of the neural network returns [logits]{.title-ref}
- raw values in \[-infty, infty\] - which are passed to the
[nn.Softmax](https://pytorch.org/docs/stable/generated/torch.nn.Softmax.html)
module. The logits are scaled to values \[0, 1\] representing the
model\'s predicted probabilities for each class. `dim` parameter
indicates the dimension along which the values must sum to 1.


In [16]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)

In [17]:
pred_probab

tensor([[0.1185, 0.0995, 0.1237, 0.0902, 0.1110, 0.0862, 0.1075, 0.0736, 0.0806,
         0.1093],
        [0.1166, 0.1013, 0.1222, 0.0925, 0.1071, 0.0860, 0.1030, 0.0752, 0.0846,
         0.1115],
        [0.1248, 0.0911, 0.1380, 0.0977, 0.1015, 0.0784, 0.0941, 0.0739, 0.0846,
         0.1159]], grad_fn=<SoftmaxBackward0>)

Model Parameters
================

Many layers inside a neural network are *parameterized*, i.e. have
associated weights and biases that are optimized during training.
Subclassing `nn.Module` automatically tracks all fields defined inside
your model object, and makes all parameters accessible using your
model\'s `parameters()` or `named_parameters()` methods.

In this example, we iterate over each parameter, and print its size and
a preview of its values.


In [18]:
print(f"Model structure: {model}\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure: NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[-0.0309,  0.0082, -0.0215,  ...,  0.0141, -0.0136, -0.0276],
        [ 0.0282,  0.0175,  0.0089,  ..., -0.0195,  0.0183,  0.0207]],
       device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([ 0.0181, -0.0051], device='cuda:0', grad_fn=<SliceBackward0>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0319,  0.0248,  0.0332,  ..., -0.0335, -0.0324,  0.0169],
        [ 0.0038,  0.0401,  0.0028,  ...,  0.0070,  0.0325,  0.0177]],
       device='cuda:0', grad_fn=<Sl

------------------------------------------------------------------------


Further Reading
===============

-   [torch.nn API](https://pytorch.org/docs/stable/nn.html)
